In [0]:
# Install packages first
!pip install qml
!pip install ase

# Lecture 2: Introduction to Machine Learning in Python, May 2020
By Dr. Anders Christensen `anders.christensen @ unibas.ch`

---

## Part 1: Non-linear regression?
Last time we saw *linear least squares regression*:


In [0]:
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [8, 6]
import numpy as np

# X-values
x = np.arange(0,20.0, 0.2)

# Y-values: Y = 1.2*X + random noise 
y = 1.2 * x + np.random.normal(scale=2.0, size=len(x))

plt.scatter(x,y)
plt.plot(x, x*1.2, color="g")

plt.show()

Approximate a target function as a weighted sum of the features:
\begin{equation}
y(\mathbf{x}) = x_1 \alpha_1 + x_2 \alpha_2 + \dots + x_n \alpha_n
\end{equation}
In matrix notation:

\begin{equation}
\mathbf{y} = \mathbf{X} \mathbf{\alpha}
\end{equation}

Minimze the error:
\begin{equation}
\mathbf{\hat{\alpha}} = \text{arg min} || \mathbf{y}^\text{ref} - \mathbf{X}\mathbf{\alpha}||^2
\end{equation}

## What a about linear regression for $\sin \left(x\right)$?

Same example as above, just with $y(x) = \sin \left(x\right)$:

In [0]:
import numpy as np
import matplotlib.pyplot as plt


x = np.arange(0,6.6, 0.6)
y = np.sin(x) #+ (np.random.random(size=len(x)) - 0.5) * 0.5

print(x.shape)
print(y.shape)

xplot = np.arange(0,6.6, 0.01)

plt.scatter(x,y, label="Training")
plt.plot(xplot, np.sin(xplot), color="g", label="sin(x)")

plt.legend()
plt.show()

Just like yesterday, we can use `numpy.linalg.lstsq()` to solve the optimal coefficients by minimizing the error:

\begin{equation}
\mathbf{\hat{\alpha}} = \text{arg min} || \mathbf{y}^\text{ref} - \mathbf{X}\mathbf{\alpha}||^2
\end{equation}
Closed-form solution:
\begin{equation}
\mathbf{\hat{\alpha}} = \left(\mathbf{X}^\top\mathbf{X} \right)^{-1}\mathbf{X}^\top\mathbf{y}^\text{ref}
\end{equation}
However, Numpy uses a singluar-value decomposition.

In [0]:
x_reshape = x.reshape(len(x),1)
alpha, sing, rank, vecs = np.linalg.lstsq(x_reshape, y, rcond=None)

print(alpha)

Making new predictions:
*   Test set of new $x$-values




In [0]:
x_test = np.arange(0.0, 6.6, 0.6)
print(x_test)
print(x_test.shape)

Predictions are made with the same equation:
\begin{equation}
\mathbf{y} = \mathbf{X} \mathbf{\alpha}
\end{equation}

In [0]:
x_test = x_test.reshape(len(x_test),1)
y_test = np.dot(x_test, alpha)

print(y_test)



Making a plot of the predictions:

In [0]:
plt.plot(xplot, np.sin(xplot), color="g", label="sin(x)")
plt.scatter(x, y, label="Training")
plt.scatter(x_test, y_test, color="r", label="Test")

plt.legend()
plt.show()

## Part 1½: Kernel ridge regression
The exercise today is about coding kernel ridge regession!

The idea is to describe the target function in terms of basis functions ("kernel functions") placed on the training points.

Plot of basis functions for the $\sin\left(x\right)$ curve:

In [0]:
import qml.kernels
import qml.math

sigma = 0.3
K = qml.kernels.gaussian_kernel(x_reshape, x_reshape, sigma)
K[np.diag_indices_from(K)] += 1e-10
alphas = qml.math.cho_solve(K, y)

xgauss = np.arange(0.0,6.0, 0.01)
fit_curve = np.zeros(len(xgauss))

for alp, xval in zip(alphas, x):
  ygauss = np.exp(-(xgauss-xval)**2/(2*sigma**2)) * alp
  fit_curve += ygauss
  # plt.plot(xgauss, ygauss, color="C0")

plt.scatter(x,y, label="Training")
plt.plot(xgauss, np.sin(xgauss), color="g", label="sin(x)")
plt.plot(xgauss, fit_curve, color="C3", label="KRR")

# plt.ylim([-1.5,1.5])
plt.legend()
plt.show()

$y\left(x\right)$ is now a sum of weighted basis functions:
\begin{equation}
\hat{y}\left(\tilde{\mathbf{x}}\right) = \sum_i \kappa\left(\mathbf{x}_i, \tilde{\mathbf{x}} \right) \alpha_i
\end{equation}

As in linear regression, $\mathbf{\alpha}_i$ are our regression weights.

In matrix form:
\begin{equation}
\mathbf{y} = \mathbf{K}\mathbf{\alpha}
\end{equation}


---



For example, Gaussian kernel:
\begin{equation}
\kappa\left(\mathbf{x}, \tilde{\mathbf{x}} \right) = \exp \left(-\frac{\|\mathbf{x} - \tilde{\mathbf{x}} \|^2}{2\sigma^2} \right)
\end{equation}
Alternative definition: (used in the exercises today)
\begin{equation}
\kappa\left(\mathbf{x}, \tilde{\mathbf{x}} \right) = \exp \left(-\gamma \|\mathbf{x} - \tilde{\mathbf{x}} \|^2 \right)
\end{equation}
The Gaussian kernel is always number between 0 and 1:
*   0 if $x$ and $\tilde{x}$ are infinitely apart
*   1 if $x$ and $\tilde{x}$ are the same


---





Fit the best $\alpha$ coefficients:

\begin{equation}
\mathbf{\hat{\alpha}} = \text{arg min} || \mathbf{y}^\text{ref} - \mathbf{K}^\mathrm{train}\mathbf{\alpha}||^2
\end{equation}
Where $\mathbf{y}^\mathrm{ref}$ are the training lables and $\mathbf{K}^\mathrm{train}$ is the pairwise kernel matrix for all the points in the training set, defined as:
\begin{equation}
\mathbf{K}^\mathrm{train}_{ij} = \kappa\left(\mathbf{x}_i, \mathbf{x}_j \right)
\end{equation}

Closed-form solution for regression coefficients:

\begin{equation}
\alpha = \left(\mathbf{K}^\mathrm{train} + \mathbf{I}\lambda\right)^{-1}\mathbf{y}^\text{ref}
\end{equation}

$\lambda$ is a small number to be added to the digonal for numerical reasons (regularization and numerical stability).


First, let's define the kernel function

In [0]:
def kernel(xi, xj):

  sigma = 0.3
  k = np.exp(-np.linalg.norm(xi - xj)**2 / (2 * sigma**2))
  
  return k

Next, let's calculate the kernel matrix:

In [0]:
K = np.zeros((len(x),len(x)))

for i in range(len(x)):
  for j in range(len(x)):
    K[i,j] = kernel(x[i], x[j])

np.set_printoptions(linewidth=666)
print(K)

With the Kernel matrix above, we can now find the regression coefficients:
\begin{equation}
\alpha = \left(\mathbf{K}^\mathrm{train} + \mathbf{I}\lambda\right)^{-1}\mathbf{y}^\text{ref}
\end{equation}


In [0]:
alphas = np.matmul(np.linalg.inv(K + np.eye(11)*1e-10), y)
print(alphas)
print(len(alphas))

Another way to solve the equation is via a so-called Cholesky decomposition which is more numerically stable:

In [0]:
import qml.math

# Add to diagonal
K[np.diag_indices_from(K)] += 1e-10

# Solve
alphas = qml.math.cho_solve(K, y)

print(alphas)


### Predictions:
Remembering again:

\begin{equation}
\hat{y}\left(\tilde{\mathbf{x}}\right) = \sum_i \kappa\left(\mathbf{x}_i, \tilde{\mathbf{x}} \right) \alpha_i
\end{equation}
In matrix form:
\begin{equation}
\mathbf{y} = \mathbf{K}\mathbf{\alpha}
\end{equation}


In [0]:
# Test points
x_test = np.random.random(size=(20,1)) * 6 
# x_test = np.arange(-5, 11.5, 0.5)


# Zero kernel
K_test = np.zeros((len(x_test),len(x)))

# Calculate pair-wise Gaussian kernel function
for i in range(len(x_test)):
  for j in range(len(x)):
    K_test[i,j] = kernel(x_test[i],  x[j])

# Make prediction
y_test = np.dot(K_test, alphas)


# Plot everything
plt.scatter(x,y, label="Training")
plt.plot(xplot, np.sin(xplot), color="g", label="True curve")
plt.scatter(x_test, y_test, color="r", label="Test")
plt.grid(True)
plt.legend()
plt.show()



## Note on Hyperparameters!

## Everything repeated, but with Scikit-learn:

Everything we've just coded is of course all available in Scikit-learn:

https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html

In [0]:
from sklearn.kernel_ridge import KernelRidge

# Gamma instead of sigma!
gam = 1 / (2.0 * sigma**2)

# Make the KRR object
krr = KernelRidge(alpha=1e-10, kernel="rbf", gamma=gam)
# krr = KernelRidge(alpha=1e-10, kernel="linear")

# Fit the machine
krr.fit(x_reshape, y)

# Prediction
y_scikit = krr.predict(x_test)

# Plot everything
plt.scatter(x,y, label="Training")
plt.plot(xgauss, np.sin(xgauss), color="g", label="True curve")
plt.scatter(x_test, y_scikit, color="r", label="Scikit-learn")

plt.legend()
plt.grid(True)
plt.show()

What about hyper parameters?

## Part 2: Molecules in and Machine Learning?






### Molecule data files

In [0]:
# Download Coordinate file for ethanol
!wget -O ethanol.xyz https://raw.githubusercontent.com/andersx/ml-intro/master/ethanol.xyz

Molecules are often stored in the ".xyz" format. For example, a conformation for ethanol is stored in the above file:

```
9

C         -0.69924       -0.01497        0.32426
C         -1.90198       -0.93192        0.56363
O          0.50350       -0.72657        0.14397
H          0.30059       -1.64320       -0.17552
H         -2.08482       -1.56542       -0.33061
H         -2.80809       -0.31867        0.75417
H         -1.71524       -1.58372        1.44281
H         -0.89079        0.62463       -0.56368
H         -0.58049        0.63584        1.21559
```

Such files can for example be visualized with ASE (Atomic Simulation Environment).
https://wiki.fysik.dtu.dk/ase/

In [0]:
!pip install ase

In [0]:
import ase.io
import ase.visualize

ethanol = ase.io.read("ethanol.xyz")
ase.visualize.view(ethanol, viewer="x3d")

# Making representations for molecules?

We need to calculate the pair wise kernels between molecules:
\begin{equation}
\kappa\left(\mathbf{x}, \tilde{\mathbf{x}} \right) = \exp \left(-\frac{\|\mathbf{x} - \tilde{\mathbf{x}} \|^2}{2\sigma^2} \right)
\end{equation}

Also sometimes called the *similarity kernel*: 1 if the two molecules are identical, 0 if they are completely different.



Problem: how do we get an input representation? Which features are good?

Anything could in principle be used:
*   Numbers of atoms
*   Physical observables
*   Name string (e.g. SMILES)
*   Coordinates
*   Nuclear charges
*   Bonding information
*   Etc ...

Some desireable properties:
*   Rotational, translations invariance?
*   Permutational invariance
*   Uniqueness (injectivity)

This kind of problem in machine learning is called "*feature engineering*".


There are *many* representations available for molecules. Two examples are follow, which will also be used in the exercies for today.

In [0]:
!pip install qml

### The Coulomb Matrix:
Proposed by Rupp et al. (2012) Phys Rev Lett https://doi.org/10.1103/PhysRevLett.108.058301

\begin{equation}
x_{ij}^\text{CM} = 
  \begin{cases}
    0.5Z_i^{2.4} & \text{for } i=j \\
    \frac{Z_i Z_j}{R_{ij}} & \text{for } i \neq j 
  \end{cases}
\end{equation}

The above takes care of translational and permutational invariance.

Furthermore, the rows and columns are sorted to ensure permutational invariance.

Example for ethane:



```
73.51669472 
34.06515641 36.8581052
19.58838249 23.5104435  36.8581052
 8.06700497  3.03799493  2.46941255  0.5
 3.91313164  5.40535191  2.78865961  0.35565997  0.5
 3.87121997  5.40076757  2.76284181  0.38595636  0.55366084  0.5
 2.9519494   2.75461693  5.40414993  0.38673507  0.39949749  0.32304269  0.5
 2.89651793  2.75224092  5.4003281   0.41811064  0.32445499  0.39915951  0.55199418  0.5
 2.35852226  2.76046537  5.40252013  0.28533493  0.40534695  0.39832789  0.5530945   0.55433763  0.5
```






Example of how to generate with QML:

In [0]:
import qml
mol = qml.Compound(xyz="ethanol.xyz")
mol.generate_coulomb_matrix(size=12)

np.set_printoptions(linewidth=100)
print(mol.representation)

### Bag-of-Bonds:
Proposed by Hansen et al. (2015) J Phys Chem Lett https://doi.org/10.1021/acs.jpclett.5b00831

Same content as the Coulomb Matrix, but items are grouped differently, so only identical terms will be compared.

More accurate for molecules? (Exercise of today)

<img src="https://i.imgur.com/eiTZ5g7.png">

In [0]:
mol.generate_bob(asize={"O":2, "C": 3, "H": 8})
np.set_printoptions(linewidth=100)
print(mol.representation)

## Exercises set #2: The QM7 dataset.
Rupp et al. (2012) Phys Rev Lett https://doi.org/10.1103/PhysRevLett.108.058301

The QM7 dataset contains XYZ structures for 7101 small molecules, with up to 7 atoms of type CNO, saturated with H.

Some of them look like this:
![alt text](https://i.imgur.com/vQymf39.png)

Attempt to map all organic molecules with up to 7 CNO-atoms.

For each molecule you will be given the atomization energy calculated using a QM method (PBE0/def2-TZVP).


*   Instead of the raw 7101 XYZ files, you will be given the Coulomb Matrix features and Bag-of-Bonds features for each molecule, along with the atomization energies.

## How to determine which machine learning method is better than the other?



For a given training/test split, one could calculate, for example, the mean-absolute-error (MAE) or root-mean-squared-error (RMSE) of the predictions, compared to the true values.

\begin{equation}
\text{MAE} =\frac{1}{N} \sum_{i=1}^{N}|y_i - \tilde{y}_i|
\end{equation}

\begin{equation}
\text{RMSE} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}\left(y_i - \tilde{y}_i\right)^2}
\end{equation}



In [0]:
y_true = np.array([0.00, 1.32, 2.64, 3.96])
y_pred = np.array([0.30, 1.14, 3.18, 2.70])

In [0]:
diff = y_true-y_pred
print(diff)

In [0]:
# Mean-absolute-error
mae = np.mean(np.abs(diff))
print(mae)

# Root-mean-squared-error
rmse = np.sqrt(np.mean(diff**2))
print(rmse)

## Learning Curves:

What is learning? Able to make better prediction with more training data.

The error decays according to a power law with the training set size:
\begin{equation}
\mathrm{Error} \propto \frac{a}{N^b}
\end{equation}
Which is the same as:
\begin{equation}
\log\left(\mathrm{Error}\right) \propto \log\left(a\right) - b \log\left(N\right)
\end{equation}
On a log-log scale the error is linear with the training set size!

It turns out, for the same dataset, these are most always parallel (same $b$-value). 

<img src="https://i.imgur.com/4xt7TMj.png" width="500">

We can compare machine learning models (for example based on different representations), by looking at the learning curve!







## Training, Test, and Validation Splits:

One common way to split data is as follows:

![alt text](https://cdn-images-1.medium.com/freeze/max/1000/1*ZiYvylk60EY2XG7ck1lqJA.png)

This is a method to avoid overfitting (i.e. fitting parameters to your "test" data)

Procedure for calculating the error for a model:
1.   Train model on **Training Set**
2.   Minimize error for **Validation Set**, i.e. optimize hyperparameters + goto 1.
3.   Calculate error for **Test Set**

The last error on the Test Set (from 3.) is your error.



Example how to split 100 values into three

*   50 Training Set
*   10 Validation Set
*   40 Test Set




In [0]:
data_set = np.arange(0,100)
print(data_set)

Remembering the Numpy *slice-notation*:

The general syntax is

`array[row]`

or

`array[row, column]`

Special notation:

    n   = what is in index n
    :n  = up to n
    n:  = n and onwards
    n:m = from n to m 
    :   = everything



In [0]:
# First 50
training_set = data_set[:50]
print(training_set)
# Next 10
validation_set = data_set[50:60]
print(validation_set)
# Next 40
test_set = data_set[60:100]
print(test_set)